## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config2 import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.29,114.16,29.44,78,68,4.92,light rain
1,1,Hithadhoo,MV,-0.60,73.08,28.35,75,89,3.50,overcast clouds
2,2,Namibe,AO,-15.20,12.15,17.50,85,25,1.51,scattered clouds
3,3,Aripuana,BR,-9.17,-60.63,26.21,37,0,0.83,clear sky
4,4,Kapaa,US,22.08,-159.32,27.00,78,75,9.30,broken clouds
5,5,Punta Arenas,CL,-53.15,-70.92,3.00,86,20,5.70,few clouds
6,6,Torbay,CA,47.67,-52.73,16.11,100,90,1.00,fog
7,7,Chuy,UY,-33.70,-53.46,7.45,85,0,4.41,clear sky
8,8,Ulladulla,AU,-35.35,150.47,9.44,70,0,4.10,clear sky
9,9,Byron Bay,AU,-28.65,153.62,15.00,62,75,5.10,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))



What is the minimum temperature you would like for your trip?20
What is the maximum temperature you would like for your trip?39


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &  (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Victoria,HK,22.29,114.16,29.44,78,68,4.92,light rain
1,1,Hithadhoo,MV,-0.60,73.08,28.35,75,89,3.50,overcast clouds
3,3,Aripuana,BR,-9.17,-60.63,26.21,37,0,0.83,clear sky
4,4,Kapaa,US,22.08,-159.32,27.00,78,75,9.30,broken clouds
10,10,Umm Lajj,SA,25.02,37.27,29.85,74,45,1.95,scattered clouds
14,14,Atuona,PF,-9.80,-139.03,26.05,74,6,8.07,clear sky
15,15,Caravelas,BR,-17.71,-39.25,21.49,81,0,2.86,clear sky
17,17,Lagos De Moreno,MX,21.35,-101.92,25.00,53,75,4.10,broken clouds
20,20,Butaritari,KI,3.07,172.79,27.92,75,62,2.76,light rain
22,22,Yellowknife,CA,62.46,-114.35,22.00,46,75,7.20,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()


City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
clean_df.count()

City_ID                383
City                   383
Country                383
Lat                    383
Lng                    383
Max Temp               383
Humidity               383
Cloudiness             383
Wind Speed             383
Current Description    383
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,29.44,light rain,22.29,114.16,
1,Hithadhoo,MV,28.35,overcast clouds,-0.60,73.08,
3,Aripuana,BR,26.21,clear sky,-9.17,-60.63,
4,Kapaa,US,27.00,broken clouds,22.08,-159.32,
10,Umm Lajj,SA,29.85,scattered clouds,25.02,37.27,
14,Atuona,PF,26.05,clear sky,-9.80,-139.03,
15,Caravelas,BR,21.49,clear sky,-17.71,-39.25,
17,Lagos De Moreno,MX,25.00,broken clouds,21.35,-101.92,
20,Butaritari,KI,27.92,light rain,3.07,172.79,
22,Yellowknife,CA,22.00,broken clouds,62.46,-114.35,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")
        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
H

In [33]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df= hotel_df[(hotel_df['Hotel Name']!="")]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Victoria,HK,29.44,light rain,22.29,114.16,Mini Hotel Central
1,Hithadhoo,MV,28.35,overcast clouds,-0.60,73.08,Scoop Guest House
4,Kapaa,US,27.00,broken clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
10,Umm Lajj,SA,29.85,scattered clouds,25.02,37.27,Moon light Furnished Units
14,Atuona,PF,26.05,clear sky,-9.80,-139.03,Villa Enata
15,Caravelas,BR,21.49,clear sky,-17.71,-39.25,Pousada dos Navegantes
17,Lagos De Moreno,MX,25.00,broken clouds,21.35,-101.92,Hotel Galerias
20,Butaritari,KI,27.92,light rain,3.07,172.79,Isles Sunset Lodge
22,Yellowknife,CA,22.00,broken clouds,62.46,-114.35,The Explorer Hotel
24,Arlit,NE,36.24,scattered clouds,18.74,7.39,Hôtel Telwa Bungalow


In [49]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [48]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))